# Perform an experiment in MLFlow
- start experiment 
- train model
- save metrics
- save artifact

In [1]:
!pip install mlflow==1.13.1 boto3 awscli -q

In [2]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature

import logging

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

warnings.filterwarnings("ignore")
np.random.seed(40)

In [3]:
data = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep=";")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
data.shape

(1599, 12)

In [5]:
TARGET_COLUMN = "quality"
train, test = train_test_split(data)

train_x = train.drop([TARGET_COLUMN], axis=1)
test_x = test.drop([TARGET_COLUMN], axis=1)
train_y = train[[TARGET_COLUMN]]
test_y = test[[TARGET_COLUMN]]

In [6]:
!aws --endpoint-url $MLFLOW_S3_ENDPOINT_URL s3 mb s3://mlflow

make_bucket failed: s3://mlflow An error occurred (503) when calling the CreateBucket operation (reached max retries: 4): Service Unavailable


In [7]:
experiment_id = mlflow.create_experiment(name="My Wine Experiments")

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): mlflow-server.kubeflow.svc.cluster.local:5000
DEBUG:urllib3.connectionpool:http://mlflow-server.kubeflow.svc.cluster.local:5000 "POST /api/2.0/mlflow/experiments/create HTTP/1.1" 200 26


In [8]:
def experiment(alpha, l1_ratio):
    mlflow.sklearn.autolog()
    with mlflow.start_run(run_name='wine_models', experiment_id=experiment_id):
            mlflow.set_tag("author", "bpk")
            lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
            lr.fit(train_x, train_y)

            pred_y = lr.predict(test_x)
            mlflow.log_metric("rmse", np.sqrt(mean_squared_error(test_y, pred_y)))
            mlflow.log_metric("r2", r2_score(test_y, pred_y))
            mlflow.log_metric("mae", mean_absolute_error(test_y, pred_y))

            signature = infer_signature(test_x, pred_y)
            mlflow.sklearn.log_model(lr, "model", registered_model_name="wine-elasticnet", signature=signature)
            
experiment(0.5, 0.5)
experiment(1, 0)
experiment(0, 1)

2023/04/05 08:25:31 INFO mlflow.utils.autologging_utils: sklearn autologging will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow to the MLflow run with ID 'f55ee609323d4b108279504850b42b25'
Successfully registered model 'wine-elasticnet'.
2023/04/05 08:25:32 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: wine-elasticnet, version 1
Created version '1' of model 'wine-elasticnet'.
2023/04/05 08:25:32 INFO mlflow.utils.autologging_utils: sklearn autologging will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow to the MLflow run with ID '247fb78a00334755b12a02d4a572370a'
Registered model 'wine-elasticnet' already exists. Creating a new version of this model...
2023/04/05 08:25:33 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model v